<a href="https://colab.research.google.com/github/santiagoprado12/Titanic---Machine-Learning-from-Disaster/blob/main/Proyecto_Final_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
install.packages("ggplot2")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [88]:
library(ggplot2)

**Subir archivos**

In [ ]:
train<-read.csv("train.csv",stringsAsFactors = FALSE)
test<-read.csv("test.csv",stringsAsFactors = FALSE)
str(train)
str(test)

**Proporcion de los sobrevivientes**

In [ ]:
table(train$Survived)
prop.table(table(train$Survived))


  0   1 
549 342 


        0         1 
0.6161616 0.3838384 

**Primera prediccion (todos mueren)**

In [ ]:
test$Survived<- rep(0,418)

submit<-data.frame(PassengerId=test$PassengerId,Survived=test$Survived)
write.csv(submit, file = "allDead.csv", row.names = FALSE)

**Porcentaje de supervivencia entre hombres y mujeres**

In [ ]:
prop.table(table(train$Survived,train$Sex),2)

   
       female      male
  0 0.2579618 0.8110919
  1 0.7420382 0.1889081

Del total de los hombres murió el 81.1% y de las mujeres murió el 25.8%

**Prediccion 2: hombres mueren, mujeres viven**

In [ ]:
test$Survived<- rep(0,418)
test$Survived[test$Sex=="female"]<-1
submit<-data.frame(PassengerId=test$PassengerId,Survived=test$Survived)
write.csv(submit, file = "allDead.csv", row.names = FALSE)

**Mujeres y niños primero!!**

In [ ]:
train$Age[is.na(train["Age"])]=mean(train$Age[!is.na(train["Age"])])
summary(train$Age)

train$Child<-0
train$Child[train$Age<18]<-1
train$Child <- factor(train$Child)

test$Child <- 0
test$Child[test$Age<18] <- 1
test$Child <- factor(test$Child)

table(train$Survived[train$Child==1])
aggregate(Survived ~ Child + Sex, data=train, FUN= function(x) {sum(x)/length(x)})
#prop.table(table(train$Survived[train$Child==1],train$Sex[train$Child==1]),2)

**Teniendo en cuenta el costo del tiquete y la clase**

In [ ]:
test$FareBin<- rep(0,418)
train$FareBin[train$Fare < 10] <- '<10'
train$FareBin[train$Fare >= 10 & train$Fare<20] <- '10-20'
train$FareBin[train$Fare >= 20] <- '20+'
table(train$FareBin)
aggregate(Survived ~ FareBin + Sex + Pclass, data=train, FUN= function(x) {sum(x)/length(x)})

test$Survived<- rep(0,418)
test$Survived[test$Sex=="female"]<-1
test$Survived[test$Pclass==3 & test$FareBin=='20+']<-0
submit<-data.frame(PassengerId=test$PassengerId,Survived=test$Survived)
write.csv(submit, file = "allDead.csv", row.names = FALSE)

**Teniendo en cuenta el titulo de la persona**

In [86]:
 train$Title<-0
 
train$Title<-sapply(train$PassengerId, FUN=function(x){strsplit(train$Name, split='[,.]')[[x]][2]})
train$Title<-sub(" ","", train$Title)
train$Title[train$Title=="Mlle"]<-"Miss"
 train$Title[train$Title=="Mme"]<-"Mrs"
train$Title[train$Title=="Lady"]<-"Miss" 
train$Title[train$Title=="Capt"]<-"Master"
train$Title[train$Title=="Don"]<-"Mr"
train$Title[train$Title=="Dr"]<-"Mr"
train$Title[train$Title=="Ms"]<-"Miss"  
train$Title[train$Title=="Jonkheer"]<-"Mr"  
train$Title[train$Title=="Sir"]<-"Mr" 
train$Title[train$Title=="Major"]<-"Master"  
train$Title[train$Title=="Col"]<-"Master"  
train$Title[train$Title=="Rev"]<-"Master"
train$Title[train$Title=="the Countess"]<-"Mrs"
table(train$Title)


Master   Miss     Mr    Mrs 
    51    186    527    127 

In [ ]:
ggplot(dataframe, aes(x = variable1, fill = factor(variable2)))+
 geom_bar(stat='count', position='dodge')+
 scale_x_continuous(breaks=c(1:valormáxeje))+
 labs(x = 'Textoetiqueta') + facet_grid(.~variable3)+
 theme_minimal()